In [1]:
import os
import numpy as np
import pandas as pd

from jmag_code import initialize_and_copy_jproj, jmag_case_input, jmag_resultscheck, initialize_jmag_app
import case_div_merge


In [2]:
class SimulationInput:
    def __init__(self):
        self.savesetting = 2
        # 0 : 샘플링 진행 아무것도 안된 처음상태
        # 1 : 초기 LHS 상태 저장 완료
        # 2 : Final sampling DATA 존재
        # 3 : 샘플링 전체 완료. serult 추출 중 정지
        
        self.opt_variable_name = ['DMAG3', 'MT1', 'MT3', 'MT4', 'MT5', 'MW1', 'MW3', 'WMAG1', 'WMAG3', 'WMAG5']
        self.opt_variable_upper = np.array([15, 14, 14, 14, 14, 40, 26, 15, 15, 20])
        self.opt_variable_lower = np.array([5, 5, 5, 5, 5, 25, 12, 5, 10, 10])

        # Efficiency Map Variable
        self.SLOTS = 36
        self.POLE = 6
        self.RPM = 4500
        self.HEIGHT = 285
        self.MECHANICAL_LOSS = 20
        self.CURRENT_SOURCE = 353.5533906
        self.COILEND_H = 110
        self.VOLTAGE_SOURCE = 3086.448943904834

        # Heat Analysis Variable
        self.AIR_HTC = 10
        self.AIR_TC = 0.02
        self.COIL_TC = 400
        self.COIL_DENS = 8.96
        self.COIL_SH = 380
        self.SHAFT_TC = 25
        self.SHAFT_DENS = 7.8
        self.SHAFT_SH = 450
        self.HOUSING_H = 420
        self.HOUSING_T = 20
        self.HOUSING_TC = 25
        self.HOUSING_DENS = 7.8
        self.HOUSING_SH = 450

        self.AMBIENT = 70

        # Load Analysis Settings
        self.OPERATING_CURRENT = 200
        self.MAGNET_TEMP = 180
        # OPERATING_PHASE는 다변수 최적화에 사용되므로 여기에 포함되지 않습니다.
        self.OPERATING_RPM = 4387
        self.OPERATING_TRQ = 923


        self.study_num = 7 #Analysis Group내의 전체 스터디 갯수

        ##### 아래 두 변수는 시퀀셜 LHS샘플링을 위한 변수들임.
        self.lhs_sample_num = 40 # 전체 샘플 갯수
        self.lhs_sample_division = int(self.lhs_sample_num/40) #sample_num을 몇개의 jmag 프로젝트 파일로 분할 할지
        ''' 즉, 한 iter에 sample_num만큼 데이터를 뽑는데 sample_division만큼의 프로젝트 파일에 
            각 sample_num/sample_division 만큼의 case를 생성시켜서 해석함.

            ex) self.lhs_sample_num = 400 / self.lhs_sample_division = 40
            한 iteration당 400개 샘플 생성 / 프로젝트 40개 / 프로젝트당 10개 case
        '''
        self.lhs_iteration_num = 20 # sampling iteration(시퀀셜)을 최대 몇번 진행할지
        
        # => 총 생성될 data는 iteration num * smple_num 임.

        #효율맵 최적화 목적함수를 위한 운전점(OP)
        self.op = [[1194, 3267],[4387, 923]] #[[rpm, torque],[rpm, torque],[rpm, torque].....]
        
        self.analysis_type_all = ['Machine Characteristics (Efficiency Map)','Cogging Torque','Torque Ripple','Induced Voltage','Demagnetization','Natural/Forced (Steady)/','Stress']
        self.analysis_type_selection = ['Machine Characteristics (Efficiency Map)','Cogging Torque','Torque Ripple','Induced Voltage']

        #Surrogate Model 선택
        self.model_types = ['linear_regression', 'polynomial_regression', 'gaussian_process', 'svm', 'decision_tree', 'random_forest', 'gradient_boosting', 'neural_network', 'xgboost', 'lightgbm', 'catboost', 'knn', 'elastic_net', 'kriging']
        
sim_input = SimulationInput()
print('Varialbe_Complete')

Varialbe_Complete


In [3]:
class Pathsetting:
    def __init__(self):
        # 현재 폴더
        self.current_dir = os.getcwd() # level 0

        # input 샘플관련 하위 폴더 명
        self.casefolder_name = '01_Generated_Case' # level 1
        self.ini_LHS_samplefolder_name = '01_1st_LHS_INsamples' # level 2
        self.seq_LHS_samplefolder_name = '02_2nd_seqLHS_INsamples' # level 2
        self.seq_divsamplefolder_name = '03_divided_INsample' # level 2
        self.sample_tempfolder = '9999999_temp' # level 2
        

        # output 샘플관련 하위 폴더 명
        self.resultfolder_name = '02_Result' # level 1
        self.LHSoutputfolder_name = '01_LHS_Result_OUTsample' # level 2
        self.results_tempfolder = '9999999_temp' # level 2

        # jproj 관련 하위 폴더 명
        self.jprojfolder_name = '03_Projectfile' # level 1
        self.divjprojfolder_name = '01_div_jproj' # level 2
        self.jproj_tempfolder = '9999999_temp' # level 2

        # 파일 명
        self.jmag_filename = '240718_EMU_Surro_OPT_After_SA.jproj'
        self.jmag_inputformat_filename = f'jmag_var_form.csv'
        self.Final_INsample_filename = f'Final_LHS_Seq_INsample.csv'  # 최종 input_seq_LHS 파일 명
        self.Final_OUTsample_filename = f'Final_LHS_Seq_OUTsample.csv'  # 최종 output_seq_LHS 파일 명

        # 주요 경로 명
        self.case_dir = os.path.join(self.current_dir, self.casefolder_name)
        self.result_dir = os.path.join(self.current_dir, self.resultfolder_name)
        self.projfolder_path = os.path.join(self.current_dir, self.jprojfolder_name)
        self.Final_INsample_filepath = os.path.join(self.case_dir, self.Final_INsample_filename)
        self.Final_OUTsample_filepath = os.path.join(self.result_dir, self.LHSoutputfolder_name, self.Final_OUTsample_filename)
        self.jmag_dir = os.path.join(self.current_dir, self.jprojfolder_name, self.jmag_filename)

pathset = Pathsetting()
print('Path Setting Complete')

Path Setting Complete


In [4]:
num_proj = 5

j_base_name, j_extension = pathset.jmag_filename.rsplit('.', 1)
c_base_name, c_extension = pathset.Final_INsample_filename.rsplit('.', 1)
r_base_name, r_extension = pathset.Final_INsample_filename.rsplit('.', 1)

divcase_folderpath = os.path.join(pathset.case_dir, pathset.seq_divsamplefolder_name)
div_jproj_folderpath = os.path.join(pathset.projfolder_path, pathset.divjprojfolder_name)

result = []
for i in range(1, num_proj + 1):
    app, jobapp = initialize_jmag_app()

    div_j_file_name = f'{j_base_name}_div{i}.{j_extension}'
    div_r_file_name = f'{r_base_name}_div{i}.{r_extension}'
    div_jproj_path = os.path.join(div_jproj_folderpath, div_j_file_name)
    div_result_path = os.path.join(pathset.result_dir, pathset.results_tempfolder, div_r_file_name)
    print('jmag_resultscheck start')
    div_result = jmag_resultscheck(app, sim_input, div_jproj_path, div_result_path)
    app.Save()
    app.Quit()
    result.append(div_result)

# Concatenate all dataframes in the output list into a single dataframe
result = pd.concat(result, ignore_index=True)

case_div_merge.case_merge(os.path.join(pathset.result_dir, pathset.results_tempfolder), os.path.join(pathset.result_dir, pathset.LHSoutputfolder_name))
#case_div_merge.delete_all_files(os.path.join(pathset.result_dir, pathset.results_tempfolder))

jmag_resultscheck start
jmag_resultscheck start


com_error: (-2147023170, '원격 프로시저를 호출하지 못했습니다.', None, None)